In [2]:
import pandas as pd

# read the csv file
df = pd.read_csv('Customer_data.csv')

df.head()


,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [3]:
# drop the 'Unnamed: 0' column
df = df.drop(columns=['Unnamed: 0'])

# convert column names to lowercase and replace spaces with underscores
df.columns = df.columns.str.lower().str.replace(' ', '_')

# convert all string values to lowercase
df = df.apply(lambda x: x.str.lower() if x.dtype == 'object' else x)

print(df.head())


  customer       state  customer_lifetime_value response  coverage education  \
0  dk49336     arizona              4809.216960       no     basic   college   
1  kx64629  california              2228.525238       no     basic   college   
2  lz68649  washington             14947.917300       no     basic  bachelor   
3  xl78013      oregon             22332.439460      yes  extended   college   
4  qa50777      oregon              9025.067525       no   premium  bachelor   

  effective_to_date employmentstatus gender  income  ...  \
0           2/18/11         employed      m   48029  ...   
1           1/18/11       unemployed      f       0  ...   
2           2/10/11         employed      m   22139  ...   
3           1/11/11         employed      m   49078  ...   
4           1/17/11    medical leave      f   23675  ...   

  number_of_open_complaints number_of_policies     policy_type        policy  \
0                       0.0                  9  corporate auto  corporate l3  

In [9]:
# drop nulls
df.dropna(inplace=True)

# convert effective_to_date to datetime
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'])

# remove duplicates
df.drop_duplicates(inplace=True)

In [10]:
# filter the dataframe
new_df = df[(df['total_claim_amount'] > 1000) & (df['response'] == 'Yes')]

In [11]:
# groupby policy type and gender and then calculate mean
average_claim = df[df['response'] == 'Yes'].groupby(['policy_type', 'gender'])['total_claim_amount'].mean()
print(average_claim)


Series([], Name: total_claim_amount, dtype: float64)


In [12]:
# count the number of customers in each state
state_counts = df['state'].value_counts()

# filter states with more than 500 customers
states_500 = state_counts[state_counts > 500]
print(states_500)

california    1552
oregon        1281
arizona        868
Name: state, dtype: int64


In [13]:
# group by education level and gender and then calculate max, min, median
grouped_df = df.groupby(['education', 'gender'])['customer_lifetime_value']
max_value = grouped_df.max()
min_value = grouped_df.min()
median_value = grouped_df.median()

print('Max: ', max_value)
print('Min: ', min_value)
print('Median: ', median_value)

Max:  education             gender
bachelor              f         58753.88046
                      m         67907.27050
college               f         61850.18803
                      m         44795.46942
doctor                f         44856.11397
                      m         32677.34284
high school or below  f         55277.44589
                      m         83325.38119
master                f         51016.06704
                      m         50568.25912
Name: customer_lifetime_value, dtype: float64
Min:  education             gender
bachelor              f         1904.000852
                      m         2030.783687
college               f         2004.350666
                      m         1918.119700
doctor                f         2395.570000
                      m         2267.604038
high school or below  f         2150.178588
                      m         2132.771675
master                f         2417.777032
                      m         2357.503128
Name

In [14]:
# convert the effective to date column to datetime
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'])

# extract month and create a new column
df['month'] = df['effective_to_date'].dt.month

# group by state and month and count the number of policies
policy_counts = df.groupby(['state', 'month']).size()

# sort the values in descending order
sorted_counts = policy_counts.sort_values(ascending=False)

# select the top 3 states
top_states = sorted_counts.head(3)

# create a new DataFrame
new_df = pd.DataFrame(top_states).reset_index()

# pivot the dataframe
pivot_df = new_df.pivot(index='state', columns='month', values=0)
print(pivot_df)


month           1      2
state                   
california  826.0  726.0
oregon      676.0    NaN
